In [1]:
from pyspark.sql import SparkSession
from delta import *
import os
from delta.tables import DeltaTable

In [2]:
spark = SparkSession.builder \
            .master("spark://spark-master:7077") \
            .config("spark.jars.packages", 
                    "org.apache.hadoop:hadoop-aws:3.2.2,"
                    "io.delta:delta-spark_2.12:3.2.0,"
                    "io.delta:delta-storage:3.2.0,"
                    "com.amazonaws:aws-java-sdk-bundle:1.12.180") \
            .config("spark.executor.memory", "4g") \
            .config("spark.executor.cores", "2") \
            .config("spark.driver.memory", "4g") \
            .config("spark.driver.cores", "2") \
            .config("spark.dynamicAllocation.enabled", "true") \
            .config("spark.dynamicAllocation.minExecutors", "1") \
            .config("spark.dynamicAllocation.maxExecutors", "2") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
	        .config("spark.hadoop.fs.s3a.endpoint", os.getenv("MINIO_HOST")) \
            .config("spark.hadoop.fs.s3a.access.key", os.getenv("MINIO_ACCESS_KEY")) \
            .config("spark.hadoop.fs.s3a.secret.key", os.getenv("MINIO_SECRET_KEY")) \
            .config("spark.hadoop.fs.s3a.path.style.access", "true") \
            .config("spark.hadoop.com.amazonaws.services.s3.enableV4", "true") \
            .config("spark.hadoop.fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
            .config("spark.hadoop.fs.AbstractFileSystem.s3a.impl","org.apache.hadoop.fs.s3a.S3A") \
            .getOrCreate()    

In [3]:
#lendo tabela
emp_path = "s3a://bronze/employe"
spark.read.format("delta").load(emp_path).createOrReplaceTempView("employe")

In [8]:
emp_path = "s3a://silver/employe_scd2"

spark.sql(f"""
CREATE TABLE IF NOT EXISTS employe_scd2 (
    emp_id INT,
    emp_name STRING,
    dept_code STRING,
    salary DOUBLE,
    start_date DATE,
    end_date DATE,
    is_current BOOLEAN
) USING DELTA LOCATION '{emp_path}'
""")

spark.sql("""
INSERT INTO employe_scd2 (emp_id, emp_name, dept_code, salary, start_date, end_date, is_current)
VALUES
    (1001, 'Alice', 'D101', 50000, '2020-01-01', NULL, TRUE),
    (1002, 'Bob', 'D102', 60000, '2020-01-01', NULL, TRUE),
    (1003, 'Charlie', 'D103', 70000, '2020-01-01', NULL, TRUE)
""")

DataFrame[]

In [6]:
spark.sql("SELECT * FROM employe_scd2").show()

+------+--------+---------+-------+----------+----------+----------+
|emp_id|emp_name|dept_code| salary|start_date|  end_date|is_current|
+------+--------+---------+-------+----------+----------+----------+
|  1001|   Alice|     D101| 2000.0|2025-03-30|      NULL|      true|
|  1002|     Bob|     D102|62000.0|2025-03-30|      NULL|      true|
|  1001|   Alice|     D101|50000.0|2020-01-01|2025-03-30|     false|
|  1002|     Bob|     D102|60000.0|2020-01-01|2025-03-30|     false|
|  1003| Charlie|     D103|70000.0|2020-01-01|      NULL|      true|
|  1006|   Frank|     D106|70000.0|2025-03-30|      NULL|      true|
|  1007|   Grace|     D107|75000.0|2025-03-30|      NULL|      true|
+------+--------+---------+-------+----------+----------+----------+



In [7]:
spark.sql("SELECT * FROM employe").show()

+------+--------+---------+-------+
|emp_id|emp_name|dept_code| salary|
+------+--------+---------+-------+
|  1006|   Frank|     D106|70000.0|
|  1007|   Grace|     D107|75000.0|
|  1001|   Alice|     D101| 1000.0|
|  1002|     Bob|     D102|62000.0|
+------+--------+---------+-------+



# SCD2

In [8]:
customersTable  = DeltaTable.forPath(spark, emp_path)  # Tabela Delta que contém os registros de empregados históricos
updatesDF  = spark.table("employe")  # DataFrame com os novos dados

In [9]:
newRecordsToInsert = updatesDF.alias("updates") \
    .join(customersTable.toDF().alias("customers"), "emp_id") \
    .where(
        "(customers.is_current = TRUE AND (updates.salary <> customers.salary OR updates.emp_name <> customers.emp_name OR updates.dept_code <> customers.dept_code))"
    )

In [10]:
newRecordsToInsert.show()

+------+--------+---------+------+--------+---------+------+----------+--------+----------+
|emp_id|emp_name|dept_code|salary|emp_name|dept_code|salary|start_date|end_date|is_current|
+------+--------+---------+------+--------+---------+------+----------+--------+----------+
|  1001|   Alice|     D101|1000.0|   Alice|     D101|2000.0|2025-03-30|    NULL|      true|
+------+--------+---------+------+--------+---------+------+----------+--------+----------+



In [11]:
# Criando o DataFrame com os registros a serem inseridos e atualizados
stagedUpdates = (
    newRecordsToInsert
    .selectExpr("NULL as mergeKey", "updates.emp_id", "updates.emp_name", "updates.dept_code", "updates.salary")  # Criamos um mergeKey (uma chave fictícia) com o valor NULL. Esse é um truque para identificar quais registros serão inseridos.
    .union(updatesDF.selectExpr("emp_id as mergeKey", "emp_id", "emp_name", "dept_code", "salary"))  # Estamos pegando as colunas de updates (dados de empregados) e preparando para a inserção. Esse conjunto de dados é para os registros novos.
)


In [12]:
stagedUpdates.show()

+--------+------+--------+---------+-------+
|mergeKey|emp_id|emp_name|dept_code| salary|
+--------+------+--------+---------+-------+
|    NULL|  1001|   Alice|     D101| 1000.0|
|    1006|  1006|   Frank|     D106|70000.0|
|    1007|  1007|   Grace|     D107|75000.0|
|    1001|  1001|   Alice|     D101| 1000.0|
|    1002|  1002|     Bob|     D102|62000.0|
+--------+------+--------+---------+-------+



In [13]:
# Aplicando a operação SCD Tipo 2 com o comando MERGE
customersTable.alias("customers").merge(
    stagedUpdates.alias("staged_updates"),
    "customers.emp_id = staged_updates.mergeKey"  # A chave de mesclagem é emp_id
) \
.whenMatchedUpdate(
    condition = "customers.is_current = true AND (customers.salary <> staged_updates.salary OR customers.emp_name <> staged_updates.emp_name OR customers.dept_code <> staged_updates.dept_code)", 
    set = {  
        "is_current": "false",  # Atualiza o campo 'is_current' para false para registros alterados
        "end_date": "current_date()"  # Define a data de término para o registro modificado
    }
) \
.whenNotMatchedInsert(
    values = {
        "emp_id": "staged_updates.emp_id",  # Insere os dados na tabela
        "emp_name": "staged_updates.emp_name",
        "dept_code": "staged_updates.dept_code",
        "salary": "staged_updates.salary",
        "start_date": "current_date()",  # Define a data de início para o novo registro
        "end_date": "NULL",  # O campo end_date para o novo registro será NULL
        "is_current": "true"  # Marca o novo registro como 'atual'
    }
) \
.execute()

In [36]:
spark.sql("DESCRIBE HISTORY employe_scd2").show(vertical=True, truncate=False)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 2                                                                                                                                                                                                                                                                                                                                 